In [67]:
import pandas as pd
import numpy as np
from scipy import stats
dataset= pd.io.stata.read_stata(r'C:\Users\jflan\Downloads\1538005773_dsc_racial_disc\EDA_racial_discrimination\data\us_job_market_discrimination.dta')

In [68]:
# 1. What test is appropriate for this problem? Does CLT apply?\n",
#    "   2. What are the null and alternate hypotheses?\n",
#    "   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.\n",
 #   "   4. Write a story describing the statistical significance in the context or the original problem
 #   "   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

In [103]:
print(dataset.keys())
print(len(dataset))

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')
4870


In [70]:
print(dataset.race.head(10))

0    w
1    w
2    b
3    b
4    w
5    w
6    w
7    b
8    b
9    b
Name: race, dtype: object


In [71]:
#1 CLT would apply as there is a large number of samples, 4870, so we assume the sample should be representative of the larger population.
#The ultimate question we are trying to solve is whether or not there is racial descrimination in the amount of people who receive a call back based on their race
# the way we can check for this is taking a percentage of white sounding names vs black sounding names and find out the differences between the two.
#The test we can use is a 2 sample t-test as we are finding whether the average difference between two groups is statistically significant 

In [72]:
#2the null hypothesis is "There is no statistical difference in call backs based on the race of the name"
#The alternative hypothesis is "There is statistically significant difference in call backs based on the race of the name"

In [73]:
#3 Let's look at the margin of error
# I will be using the confidence level 95%. First I need to find the variance for both populations
# Let's start with the numbers of people who were sampled that were black sounding/ received called

black_total=sum(dataset.race=='b')
print(black_total)
black_call=sum(dataset[dataset.race=='b'].call)
print(black_call)

2435
157.0


In [74]:
#same method for white sounding names
white_total= sum(dataset.race=='w')
print(white_total)
white_call=sum(dataset[dataset.race=='w'].call)
print(white_call)

2435
235.0


In [75]:
# percentages for both
p_black_called=black_call/white_total
p_white_called=white_call/black_total
print(p_black_called,p_white_called)

0.06447638603696099 0.09650924024640657


In [76]:
#black callback variace
variance_black=(p_black_called*(1-p_black_called)/black_total)
print(variance_black)

2.4771737856498466e-05


In [77]:
#white callback variance
variance_white=(p_white_called*(1-p_white_called)/white_total)
print(variance_white)

3.580911983304638e-05


In [78]:
#sampling distribution for variances
variance=variance_black+variance_white

In [79]:
#calc margin of error
margin_error=np.sqrt(variance)*1.96
print(margin_error)

0.015255406349886438


In [83]:
#check if it's within sampling dist variance. COME BACK TO THIS!!!
dif=abs(p_black_called-p_white_called)
min_dif=dif-margin_error
max_dif=dif+margin_error
print(min_dif,max_dif,dif)

0.016777447859559147 0.047288260559332024 0.032032854209445585


In [100]:
#calculating the degrees of freedom
black_df=variance_black/black_total
white_df=variance_white/white_total
df=((black_df+white_df)**2)/(((black_df**2)/black_total)+((white_df**2)/white_total))
print(df)

4713.53819343226


In [87]:
#lets work out the t/ p values
standard_e= np.sqrt(variance)
print(standard_e)

0.0077833705866767544


In [89]:
t_value=((p_black_called-p_white_called)-0)/standard_e
print(t_value)

-4.11555043573


In [92]:
p_value=stats.t.sf(np.abs(t_value),df)*2
print(p_value)

3.883831746581082e-05


In [97]:
black=dataset[dataset.race=='b'].call
white=dataset[dataset.race=='w'].call

In [99]:
stats.ttest_ind(black,white,equal_var=False)

Ttest_indResult(statistic=-4.114705290861751, pvalue=3.942941513645935e-05)

In [ ]:
#Q4
#p value is less than .05 so we will reject the null hypothesis. This means that there is a statistically significant difference in the effect black and white sounding names have on callbacks
#Per our confidence interval we can say that we are 95% condfident that a black or white sounding name could influence the rate of callback by 1.7-4.7%. This is a major issue considering how few people received callbacks <10%
#q5 I would assume that race is not the most important factor considering it's a small margin of difference in an already small margin. We can assume that there are other reasons that 90% of applicants didn't receive calls in the test. Considering the fact that there are 65 different variables each variable should be tested and ranked on which correlated with a higher call rate. 
#This could get us closer to figuring out the entire story of causation 
